# Image segmentation

The task is to train a simple deep learning model to segment the cells visible in wide fluorescence images.
For this I use the <b>'tensorflow.keras'</b> library because it is quite easy to use and provides all the necessary tools.

I downloaded the images and put them in folders right next to the Python program. \
The folders are called <b>'img_seg'</b> (the masks) and <b>'img_full'</b> (the normal images). 

```
/ 
+--img_seg 
+--img_full 
+--Image_Segmentation.ipynb
```

Each dataset consisted of wide-field epifluorescence images of cultured neurons with both cytoplasmic (phalloidin) and nuclear staining (DAPI) and a set of manual segmentations of the neuronal and nuclear boundaries.

As far as I can tell, the sets of manual segmentations, the masks, are all from the images with the cytoplasmic (phalloidin) channel. 


<table><tr>
<td><img src="https://cildata.crbs.ucsd.edu/display_images/ccdb/ccdb_512/6843_512r.jpg" width="250px"></td>
<td><img src="https://cildata.crbs.ucsd.edu/display_images/ccdb/ccdb_512/6843_512s.jpg" width="250px"></td>
</tr></table>


therefore I decided to only use them to train my model.

Since only some of the wide-field epifluorescence images have more than one corresponding mask, i also decided to use only one mask for each image. This still leaves me with 100 images and 100 masks. 

In [8]:
import tensorflow.keras as tf_k                              #This library is for modeling the network.


import os                                                    # This two librarys are used to load
import matplotlib.image as img                               # the data from the folders into the Program


import numpy as np                                           # To reshape and to normalize the data,
from skimage.transform import downscale_local_mean as dlm    # i use this two librarys.

import matplotlib.pyplot as plt                              # And this library is for ploting the results

%matplotlib inline  





The function Looks through both folders and loads only the images that are needed.

In [9]:
def load_training_images(foldername_1='img_seg', foldername_2='img_full'):
    masks = []
    images = []
    
    for filename in os.listdir(foldername_1):
        if "GT_01" in filename:
            masks.append(  img.imread(  os.path.join(foldername_1, filename))[:,:,0]   )
        else:
            pass

    for filename in os.listdir(foldername_2):
        if "w1" in filename:
            images.append( dlm( img.imread(  os.path.join(foldername_2, filename)  ), (2,2) )  )
        else:
            pass
    return masks, images


load_y, load_x = load_training_images()





Normalize data by dividing all values by the highest value. \
I use the highest value of all the images, not just the current one.

In [10]:
def Normalize(Liste):
    max_value = np.max(Liste)
    for k in range(len(Liste)):
        Liste[k] = Liste[k]/max_value
    return Liste

norm_y_train = Normalize(load_y)
norm_x_train = Normalize(load_x)





Divides the data in training and in testing data, and brings them in the right shape \
The percentage attribute is the ratio of training and test data. By default the ratio is 70 to 30.

In [11]:
def get_data(masks, images, percentage=70):
    
    masks_train = np.array(masks).reshape(100,520,696,1)[:percentage]
    images_train = np.array(images).reshape(100,520,696,1)[:percentage]
    masks_test = np.array(masks).reshape(100,520,696,1)[percentage:100]
    images_test  = np.array(images).reshape(100,520,696,1)[percentage:100]
    
    return masks_train, images_train, masks_test, images_test


y_train, x_train, y_test, x_test = get_data(norm_y_train, norm_x_train)

### Network architecture

The architecture im uesing is a normal <b> U-Net architecture </b>, which i made a bit smaller than it is usually. \
I did this becaus becaus i got performanc problems while running the program. So i ended up uesing only two <b> channe sizes, 8 and 16 </b>. 

The shape of the data goes from <b>520x696x1</b> to <b>260x348x8</b> to <b>130x174x16</b> and then back again.

I also use a <b>dropout of 30%</b> to prevent overfitting.

In [12]:
#The U-Net:


def get_model(chanels=[8, 16], drop_rate=0.3):

    inputs = tf_k.Input(shape=(520,696,1))                                       

    c1 = tf_k.layers.Conv2D(chanels[0], 3, activation='relu', padding="same")(inputs)
    c1 = tf_k.layers.Dropout(drop_rate)(c1)
    c1 = tf_k.layers.Conv2D(chanels[0], 3, activation='relu', padding="same")(c1)

    p1 = tf_k.layers.MaxPool2D((2,2))(c1)                                        

    c2 = tf_k.layers.Conv2D(chanels[1], 3, activation='relu', padding="same")(p1)
    c2 = tf_k.layers.Dropout(drop_rate)(c2)
    c2 = tf_k.layers.Conv2D(chanels[1], 3, activation='relu', padding="same")(c2)  

    p2 = tf_k.layers.MaxPool2D((2,2))(c2)                                        

    c3 = tf_k.layers.Conv2D(chanels[1], 3, activation='relu', padding="same")(p2)
    c3 = tf_k.layers.Dropout(drop_rate)(c3)
    c3 = tf_k.layers.Conv2D(chanels[1], 3, activation='relu', padding="same")(c3)

    u1 = tf_k.layers.Conv2DTranspose(16, 2, strides=2, padding="same")(c3)
    u1 = tf_k.layers.concatenate([c2,u1])

    c4 = tf_k.layers.Conv2D(chanels[1], 3, activation='relu', padding="same")(u1)
    c4 = tf_k.layers.Dropout(drop_rate)(c4)
    c4 = tf_k.layers.Conv2D(chanels[1], 3, activation='relu', padding="same")(c4)

    u2 = tf_k.layers.Conv2DTranspose(chanels[0], 2, strides=2, padding="same")(c4)
    u2 = tf_k.layers.concatenate([c1,u2])

    c5 = tf_k.layers.Conv2D(chanels[0], 3, activation='relu', padding="same")(u2)
    c5 = tf_k.layers.Dropout(drop_rate)(c5)
    c5 = tf_k.layers.Conv2D(chanels[0], 3, activation='relu', padding="same")(c5)


    outputs = tf_k.layers.Conv2D(1, 1, activation='sigmoid', padding="same")(c5) 
    
    return tf_k.Model(inputs=inputs, outputs=outputs)
    

In [ ]:
my_model = get_model()
my_model.compile(optimizer="Adam", loss="BinaryCrossentropy", metrics=["accuracy"])
my_model.fit(x_train, y_train, epochs=3, batch_size=5)

Epoch 1/3
14/14 [==============================] - 46s 3s/step - loss: 0.6765 - accuracy: 0.7420
Epoch 2/3
14/14 [==============================] - 45s 3s/step - loss: 0.5656 - accuracy: 0.7705
Epoch 3/3
 3/14 [=====>........................] - ETA: 35s - loss: 0.3755 - accuracy: 0.8738

In [ ]:
my_model.evaluate(x_test, y_test, batch_size=5)

In [ ]:
index = 8
predictions = my_model(x_test[index:index+1]).numpy()
predictions2 = my_model(x_train[index:index+1]).numpy()

fig, ax = plt.subplots(2,3, figsize=(20,10))
ax[0][0].imshow(x_train[index])
ax[0][1].imshow(predictions2[0])
ax[0][2].imshow(y_train[index])


ax[1][0].imshow(x_test[index])
ax[1][1].imshow(predictions[0])